In [1]:
from itertools import product
from math import sin, cos
import numpy as np
from z3 import *

In [2]:
V_STOP, V_LOW, V_HIGH = 0, 1, 10 # velocidades dos modos baixo e alto, em m
L = 1e3 # Lado dos setores em m
THETA = 15 # angulo de viragem entre modos
TAU = 3 # tempo minimo entre transicoes timed
ALL_ROUTES = [i*THETA for i in range(int(360/THETA))] # todos os ângulos possíveis
ALL_VELS = [V_STOP, V_LOW, V_HIGH] # todas as velocidades possiveis

Mode, (M_STOP, M_LOW, M_HIGH) = EnumSort("Mode", ("V_STOP", "V_LOW", "V_HIGH"))

In [3]:
deg_to_rad = lambda a: a * np.pi / 180
vel_to_mode = lambda v: M_LOW if (v == V_LOW) else M_HIGH if (v == V_HIGH) else M_STOP
mode_to_vel = lambda m: V_LOW if (m == M_LOW) else V_HIGH if (m == M_HIGH) else V_STOP

In [4]:
def declare(i, num_boats=3):
    trace = {}
    for j in range(num_boats):
        trace[f"b{j}"] = {}
        trace[f"b{j}"]["v"] = Const(f"b{j}_v{i}", Mode)
        trace[f"b{j}"]["a"] = Int(f"b{j}_a{i}")
        trace[f"b{j}"]["sx"] = Int(f"b{j}_sx{i}")
        trace[f"b{j}"]["sy"] = Int(f"b{j}_sy{i}")
        trace[f"b{j}"]["x"] = Real(f"b{j}_x{i}")
        trace[f"b{j}"]["y"] = Real(f"b{j}_y{i}")
        trace[f"b{j}"]["t"] = Real(f"b{j}_t{i}")
        
    trace["c"] = {}
    trace["c"]["v"] = Const("c_v", Mode)
    trace["c"]["a"] = Int("c_a")
    trace["c"]["sx"] = Int("c_sx")
    trace["c"]["sy"] = Int("c_sy")
    
    return trace
        
        
def init(trace, N):
    r = []
    for b in trace:
        if "b" in b:
            # Inicializar cada um dos barcos
            r.append(trace[b]["v"] == M_HIGH)
            index = np.random.randint(len(ALL_ROUTES))
            r.append(trace[b]["a"] == ALL_ROUTES[index])
            r.append(trace[b]["sx"] == np.random.randint(N))
            r.append(trace[b]["sy"] == np.random.randint(N))
            r.append(And(trace[b]["x"] >= L*trace[b]["sx"], trace[b]["x"] <= L*(trace[b]["sx"]+1)))
            r.append(And(trace[b]["y"] >= L*trace[b]["sy"], trace[b]["y"] <= L*(trace[b]["sy"]+1)))
            r.append(trace[b]["t"] == 0)
            
    # Inicializar o controlador com o estado um barco aleatório
    index = np.random.randint(len(trace)-1)
    r.append(trace["c"]["v"] == trace[f"b{index}"]["v"])
    r.append(trace["c"]["a"] == trace[f"b{index}"]["a"])
    r.append(trace["c"]["sx"] == trace[f"b{index}"]["sx"])
    r.append(trace["c"]["sy"] == trace[f"b{index}"]["sy"])
            
    return And(r)

## Transições das variáveis contínuas

In [5]:
def update_pos(prev, curr, b):
    r = []
    modes = list(product(*[ALL_ROUTES, ALL_VELS]))
    
    for v, a in modes:
        vel = prev[b]["v"] == vel_to_mode(v)
        ang = prev[b]["a"] == a
        x = curr[b]["x"] == prev[b]["x"] + v * cos(deg_to_rad(a)) * (curr[b]["t"]-prev[b]["t"])
        y = curr[b]["y"] == prev[b]["y"] + v * sin(deg_to_rad(a)) * (curr[b]["t"]-prev[b]["t"])
        r.append(And(x, y, ang, vel))
    
    return Or(r)

def high_high(prev, curr, b):
    r = []
    
    r.append(And(prev[b]["v"] == M_HIGH, curr[b]["v"] == M_HIGH))
    r.append(curr[b]["t"] > prev[b]["t"])
    r.append(update_pos(prev, curr, b))
    
    return And(r)

def high_low(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_HIGH, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] == prev[b]["t"] + 500)
    r.append(update_pos(prev, curr, b))
    
    return And(r)
    
def low_high(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_HIGH))
    r.append(curr[b]["t"] == prev[b]["t"] + 500)
    r.append(update_pos(prev, curr, b))
    
    return And(r)

def low_low(prev, curr, b):
    r = []
    
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] > prev[b]["t"])
    r.append(update_pos(prev, curr, b))
    
    return And(r)
    
def low_stop(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_LOW, curr[b]["v"] == M_STOP))
    r.append(curr[b]["t"] == prev[b]["t"] + 50)
    r.append(update_pos(prev, curr, b))
    
    return And(r)
    
def stop_low(prev, curr, b):
    r = []
    
    # TODO: adicionar resticoes a rota
    r.append(And(prev[b]["v"] == M_STOP, curr[b]["v"] == M_LOW))
    r.append(curr[b]["t"] == prev[b]["t"] + 50)
    r.append(update_pos(prev, curr, b))
    
    return And(r)

def stop_stop(prev, curr, b):
    r = []
    
    r.append(And(prev[b]["v"] == M_STOP, curr[b]["v"] == M_STOP))
    r.append(curr[b]["t"] > prev[b]["t"])
    r.append(update_pos(prev, curr, b))
    
    return And(r)

## Untimed

Considerações:

- Quando um barco muda de setor e não está em perigo, ele continua no mesmo modo

In [6]:
def danger_boats(trace, b1, b2):
    sx = And(trace[b1]["sx"]<=trace[b2]["sx"]+1, trace[b1]["sx"]>=trace[b2]["sx"]-1)
    sy = And(trace[b1]["sy"]<=trace[b2]["sy"]+1, trace[b1]["sy"]>=trace[b2]["sy"]-1)
    return And(sx, sy)

danger = lambda trace, b: Or([danger_boats(trace, b_, b) for b_ in trace if ("b" in b_ and b!=b_)])
sector_change = lambda prev, curr, b: Or(prev[b]["sx"]!=curr[b]["sx"], prev[b]["sy"] != curr[b]["sy"])

def b_untimed(prev, curr, b):
    # Mudança das variáveis contínuas do barco
    low2stop, low2low, low2high = low_stop(prev, curr, b), low_low(prev, curr, b), low_high(prev, curr, b)
    high2low, high2high = high_low(prev, curr, b), high_high(prev, curr, b)
    xyt_evolve = Or(low2stop, low2low, low2high, high2low, high2high)
    
    # Mudança do setor do barco
    sx = curr[b]["sx"] == curr[b]["x"] / L
    sy = curr[b]["sy"] == curr[b]["y"] / L
    sector = And(sx, sy)
    
    # Mundaça de modo do barco de acordo com o controlador
    change = sector_change(prev, curr, b)
    vel_ang = And(curr[b]["v"] == curr["c"]["v"], curr[b]["a"] == curr["c"]["a"])
    
    return And(xyt_evolve, sector, change, vel_ang)

In [7]:
def adjacent_boats(trace, b1, b2):
    left  = And(trace[b1]["sx"]==trace[b2]["sx"]-1, trace[b1]["sy"]==trace[b2]["sy"])
    right = And(trace[b1]["sx"]==trace[b2]["sx"]+1, trace[b1]["sy"]==trace[b2]["sy"])
    up    = And(trace[b1]["sx"]==trace[b2]["sx"], trace[b1]["sy"]==trace[b2]["sy"]+1)
    down  = And(trace[b1]["sx"]==trace[b2]["sx"], trace[b1]["sy"]==trace[b2]["sy"]-1)
    
    return Or (left, right, up, down)

adjacent = lambda trace, b: Or([adjacent_boats(trace, b, b_) for b_ in trace if ("b" in b_ and b!=b_)])

def c_untimed(prev, curr):
    r = []
    for b in prev:
        if "b" in b:
            # Mudança do setor do controlador
            change = sector_change(prev, curr, b)
            sx = curr["c"]["sx"] == curr[b]["sx"]
            sy = curr["c"]["sy"] == curr[b]["sy"]
            sector = And(change, sx, sy)
            
            # TODO: fix desta parte
            # Mudança da velocidade do controlador
            b_danger = And(danger(curr, b), curr["c"]["v"]==M_LOW)
            b_adj = And(adjacent(curr, b), And(curr["c"]["v"]==M_STOP))
            b_safe = And(Not(danger(curr, b)), curr["c"]["v"]==M_HIGH)
            v = Or(b_danger, b_safe)
            
            r.append(And(sector, v))
            
    return Or(r)

untimed = lambda prev, curr: And([b_untimed(prev, curr, b) for b in prev if "b" in b] + [c_untimed(prev, curr)])

## Timed

In [8]:
def b_timed(prev, curr, b):
    # Não ocorre mudanças de modo
    v = prev[b]["v"] == curr[b]["v"]
    a = prev[b]["a"] == curr[b]["a"]
    sx = prev[b]["sx"] == curr[b]["sx"]
    sy = prev[b]["sy"] == curr[b]["sy"]
    
    # Mudança das variáveis contínuas
    stop = stop_stop(prev, curr, b) 
    low = low_low(prev, curr, b)
    high = high_high(prev, curr, b)
    
    return And(And(v, a, sx, sy), Or(stop, high, low))

In [9]:
def c_timed(prev, curr):
    # Não ocorre nenhuma mudança no controlador
    v = prev["c"]["v"] == curr["c"]["v"]
    a = prev["c"]["a"] == curr["c"]["a"]
    sx = prev["c"]["sx"] == curr["c"]["sx"]
    sy = prev["c"]["sy"] == curr["c"]["sy"]
    
    return And(v, a, sx, sy)

timed = lambda prev, curr: And([b_timed(prev, curr, b) for b in prev if "b" in b] + [c_timed(prev, curr)])

In [10]:
def mixed(prev, curr):
    uut = And(b_untimed(prev, curr, "b0"), b_untimed(prev, curr, "b1"), b_timed(prev, curr, "b2"))
    utu = And(b_untimed(prev, curr, "b0"), b_timed(prev, curr, "b1"), b_untimed(prev, curr, "b2"))
    tuu = And(b_timed(prev, curr, "b0"), b_untimed(prev, curr, "b1"), b_untimed(prev, curr, "b2"))
    utt = And(b_untimed(prev, curr, "b0"), b_timed(prev, curr, "b1"), b_timed(prev, curr, "b2"))
    tut = And(b_timed(prev, curr, "b0"), b_untimed(prev, curr, "b1"), b_timed(prev, curr, "b2"))
    ttu = And(b_timed(prev, curr, "b0"), b_timed(prev, curr, "b1"), b_untimed(prev, curr, "b2"))
    
    return And(Or(uut, utu, tuu, utt, tut, ttu), c_untimed(prev, curr))

In [11]:
def eqd(prev, curr):
    t1 = curr["b0"]["t"]-prev["b0"]["t"] == curr["b1"]["t"]-prev["b1"]["t"]
    t2 = curr["b1"]["t"]-prev["b1"]["t"] == curr["b2"]["t"]-prev["b2"]["t"]
        
    return And(t1, t2)

In [12]:
trans = lambda prev, curr: And(eqd(prev, curr), Or(untimed(prev, curr), timed(prev, curr), mixed(prev, curr)))

In [13]:
num_sectors = 10

trace0, trace1 = declare(0), declare(1)
solver = Solver()
solver.add(init(trace0, num_sectors))
solver.add(trans(trace0, trace1))

if solver.check() == sat:
    print("Dere is a we")
else:
    print("No we")

No we
